In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from tqdm.notebook import tqdm
import seaborn as sns
import re 

In [3]:
if not os.getcwd().endswith('help_bot'):
    os.chdir('/home/payonear/ods_help_bot')

### Ideas:
* filer channels
* filer removed messages
* filter subtype
* filter trash (like only punctuations)
* filter short texts
* filter those with negative reactions

In [24]:
data = pd.read_csv('./data/ods_slack_all.csv', low_memory=False)
data.head(3)

,Unnamed: 0,type,subtype,ts,user,text,channel,old_name,name,reactions,...,is_intro,x_files,username,icons,bot_link,room,no_notifications,permalink,bot_profile,text_len
0,4,message,channel_name,1.534698e+09,U040HKJE7,"<@U040HKJE7> has renamed the channel from ""_to...",___top_links,_top_links,___top_links,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72
1,5,message,channel_archive,1.534698e+09,U040HKJE7,<@U040HKJE7> archived the channel,___top_links,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33
2,9,message,channel_archive,1.476872e+09,U040HKJE7,<@U040HKJE7> archived the channel,_call_4_collaboratio,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33


In [5]:
data.shape

(1373439, 52)

In [6]:
data.columns

Index(['Unnamed: 0', 'type', 'subtype', 'ts', 'user', 'text', 'channel',
       'old_name', 'name', 'reactions', 'client_msg_id', 'team', 'user_team',
       'source_team', 'user_profile', 'blocks', 'thread_ts', 'parent_user_id',
       'attachments', 'reply_count', 'reply_users_count', 'latest_reply',
       'reply_users', 'replies', 'subscribed', 'edited', 'last_read', 'files',
       'upload', 'display_as_bot', 'hidden', 'upload_reply_to', 'root',
       'inviter', 'new_broadcast', 'topic', 'item_type', 'purpose', 'item',
       'bot_id', 'file', 'comment', 'is_intro', 'x_files', 'username', 'icons',
       'bot_link', 'room', 'no_notifications', 'permalink', 'bot_profile',
       'text_len'],
      dtype='object')

## Filter channels

In [25]:
chan = [
    '___top_links', 'analytiсs', 'analytiсs', 'article_essence', 'audio_and_speech','bayesian', 'big_data', 'bioinformatics' \
    'blockchain', 'bots', 'business', 'career', 'class_coursera_aml', 'class_cs224n', 'class_cs224w', 'class_cs231n', 'class_dl_cmu', \
    'class_dl_mipt', 'class_fast_ai', 'class_fastai', 'class_ium_2019','class_udacity_pytorch', 'cloud', 'conference', 'consulting',\
    'coronavirus', 'cv','data_fest', 'data_labeling', 'datasets', 'deep', 'deep_learning','deephack', 'dev_bot_test', 'devops', 'dlcourse_ai',\
    'ds_process', 'edu_academy', 'edu_books', 'edu_course_pgm', 'edu_courses','edu_dsclub_spb', 'edu_shad_exams', 'engineering-hiring', 'esports',\
    'fnbic_neural_networks', 'gan', 'gis', 'gnomiki', 'hardware', 'huawei_nlp_course', 'industrial_ds', 'interesting_links', 'interpretable_ml',\
    'kaggle_crackers', 'lang_cpp', 'lang_go', 'lang_javascript', 'lang_julia', 'lang_lua_torch', 'lang_python', 'lang_r', 'lang_rust', 'lang_scala',\
    'math', 'medicine', 'ml4sg', 'ml_pipeline', 'mlart', 'mlcourse_ai', 'mlcourse_ai_news', 'mlcourse_ai_rus', 'mlcourse_dubai', 'mlrepa', \
    'mltrainings_beginners', 'mltrainings_live', 'network_analysis','neuroscience', 'nlp', 'ods_code_review', 'ods_crowd', 'ods_datasets',\
    'ods_editors', 'ods_habr', 'ods_mentoring', 'ods_merch', 'ods_pet_projects', 'ods_platform', 'ods_podcast', 'ods_policy_prior','ods_resume_mastering',\
    'optimization','pr_jerk', 'proj_acm_recsys', 'proj_algo_alice', 'proj_benchr','proj_bytecup2018', 'proj_classic_ai', 'proj_comp_workshop',\
    'proj_covid', 'proj_dota2', 'proj_drivendata','proj_drivendata_chimp', 'proj_ds_bowl2018', 'proj_dsbowl_2017','proj_dsbowl_2019', 'proj_fakenewschalleng', \
    'proj_freesound_2019','proj_gensim', 'proj_geohack', 'proj_gold_miners','proj_home_credit', 'proj_idao_2019', 'proj_jet_raif2018','proj_kaggle_2sigma', \
    'proj_kaggle_abstraction_and_reasoning', 'proj_kaggle_airbus', 'proj_kaggle_alaska', 'proj_kaggle_avito', 'proj_kaggle_bengali_classification', \
    'proj_kaggle_birds', 'proj_kaggle_blindness', 'proj_kaggle_camera', 'proj_kaggle_cars','proj_kaggle_cdiscount', 'proj_kaggle_cervix','proj_kaggle_champs', \
    'proj_kaggle_covid_vaccine', 'proj_kaggle_deepfake', 'proj_kaggle_elo_merch', 'proj_kaggle_energy', 'proj_kaggle_fashion', 'proj_kaggle_fgvc6',\
    'proj_kaggle_fish', 'proj_kaggle_fraud', 'proj_kaggle_ga','proj_kaggle_gan', 'proj_kaggle_halite', 'proj_kaggle_hemorrhage','proj_kaggle_inclusive', \
    'proj_kaggle_inst_grat', 'proj_kaggle_instacart', 'proj_kaggle_ion', 'proj_kaggle_jigsaw','proj_kaggle_landmark', 'proj_kaggle_lanl', 'proj_kaggle_lyft',\
    'proj_kaggle_m5', 'proj_kaggle_mechanism_of_action', 'proj_kaggle_melanoma', 'proj_kaggle_nfl', 'proj_kaggle_openimage','proj_kaggle_petfinder', \
    'proj_kaggle_plasticc','proj_kaggle_pneumonia', 'proj_kaggle_pronoun','proj_kaggle_protein', 'proj_kaggle_quest_challenge','proj_kaggle_quickdraw', \
    'proj_kaggle_quora_qs','proj_kaggle_recruit', 'proj_kaggle_salt', 'proj_kaggle_santa','proj_kaggle_santander', 'proj_kaggle_severstal','proj_kaggle_speech',\
    'proj_kaggle_statoil','proj_kaggle_talkdata', 'proj_kaggle_tf2_question_ans','proj_kaggle_trackml', 'proj_kaggle_trends_neuroimaging',\
    'proj_kaggle_tweet_sent_extr', 'proj_kaggle_vsb_power','proj_kaggle_wheat_detection', 'proj_kekas', 'proj_mercari','proj_msft_malware', \
    'proj_nexar2', 'proj_nghack2019','proj_nips_autodl', 'proj_nips_causal', 'proj_nips_comp_19','proj_nips_comp_20', 'proj_nips_traffic', \
    'proj_ods_introspect', 'proj_poker', 'proj_procurement_shr', 'proj_prostate_cancer','proj_punch_to_face', 'proj_restrserve', 'proj_retailhero',\
    'proj_rosbank', 'proj_safe_driver', 'proj_sibur_challenge', 'proj_signate', 'proj_signate_cars', 'proj_snahackathon','proj_telecom_data_cup', \
    'proj_tinkoff', 'proj_topcoder','proj_waymo_challenges', 'proj_wildfire', 'proj_yt8m','proj_zillow', 'proj_zindi',  'pydata', 'quantum_computing',\
    'recommender_systems','reinforcement_learning', 'risk_modelling', 'rnn','satellite_imaging','security','self_driving', 'sequences_series',\
    'show_your_work', 'spark', 'stack_overflow', 'teal_practices_cases', 'theory_and_practice', 'trading', 'visualization','whateverhack', 'work',\
    'writing_n_presenting'
] 

In [26]:
print(f'Total channels: {len(data.channel.unique())}, removed: {len(data.channel.unique()) - len(chan)}.')

Total channels: 348, removed: 124.


In [27]:
data = data[data['channel'].isin(chan)]

In [10]:
data.shape

(875862, 52)

## Filter removed

In [28]:
data = data[data['text']!='This message was deleted.']

In [29]:
data.shape

(874655, 52)

## Filter subtype

In [11]:
data.head(3)

,Unnamed: 0,type,subtype,ts,user,text,channel,old_name,name,reactions,...,is_intro,x_files,username,icons,bot_link,room,no_notifications,permalink,bot_profile,text_len
0,4,message,channel_name,1.534698e+09,U040HKJE7,"<@U040HKJE7> has renamed the channel from ""_to...",___top_links,_top_links,___top_links,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72
1,5,message,channel_archive,1.534698e+09,U040HKJE7,<@U040HKJE7> archived the channel,___top_links,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33
430669,639944,message,NaN,1.585149e+09,UDD345XLG,"Исследователи из Стэнфорда обнаружили, что сам...",ai_ethics,NaN,NaN,"[{'name': 'notbad', 'users': ['UC9HBTUKG', 'UF...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,371


In [30]:
data.subtype.unique()

array(['channel_name', 'channel_archive', nan, 'channel_purpose',
       'thread_broadcast', 'channel_topic', 'file_comment', 'pinned_item',
       'channel_leave', 'bot_add', 'bot_remove', 'bot_message',
       'sh_room_created', 'reminder_add', 'app_conversation_leave',
       'channel_unarchive', 'me_message', 'tombstone'], dtype=object)

In [31]:
stype = ['channel_name', 'channel_archive', 'channel_topic', 'channel_purpose', 'file_comment', 'pinned_item',
       'channel_leave', 'bot_add', 'bot_remove', 'bot_message', 'sh_room_created', 'reminder_add', 'app_conversation_leave',
       'channel_unarchive', 'me_message']

In [32]:
data = data[~data['subtype'].isin(stype)]

In [33]:
data.shape

(844304, 52)

## Filter out short channels

In [22]:
# data.channel.value_counts(ascending=True)

In [34]:
chan = ['stack_overflow',
'mlrepa',
'dev_bot_test',
'proj_nips_autodl',
'proj_nips_traffic',
'ods_datasets',
'proj_nips_comp_20',
'proj_kaggle_halite',
'proj_kaggle_covid_vaccine',
'data_labeling',
'esports',
'mlart',
'fnbic_neural_networks',
'class_ium_2019',
'ods_podcast',
'ods_editors',
'class_dl_mipt',
'proj_idao_2019',
'proj_snahackathon',
'lang_rust',
'whateverhack',
'proj_jet_raif2018',
'proj_bytecup2018',
'proj_acm_recsys',
'proj_sibur_challenge',
'proj_signate']

In [35]:
data = data[~data['channel'].isin(chan)]
data.shape

(843890, 52)

## Merge questions and answers

In [36]:
data.index = data['user'] + '_' + data['ts'].round(0).astype(str)
data['new_ind'] = data.index

In [17]:
data.head()

,Unnamed: 0,type,subtype,ts,user,text,channel,old_name,name,reactions,...,x_files,username,icons,bot_link,room,no_notifications,permalink,bot_profile,text_len,new_ind
UDD345XLG_1585149097.0,639944,message,NaN,1.585149e+09,UDD345XLG,"Исследователи из Стэнфорда обнаружили, что сам...",ai_ethics,NaN,NaN,"[{'name': 'notbad', 'users': ['UC9HBTUKG', 'UF...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,371,UDD345XLG_1585149097.0
U4Z9Q18QH_1585149345.0,639945,message,NaN,1.585149e+09,U4Z9Q18QH,а поправку на уровень образования делали? Увер...,ai_ethics,NaN,NaN,"[{'name': 'kekeke', 'users': ['UDD345XLG'], 'c...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95,U4Z9Q18QH_1585149345.0
U34Q3KU8H_1585152153.0,639946,message,NaN,1.585152e+09,U34Q3KU8H,"Я их тоже хуже понимаю. Многие говорят, как бу...",ai_ethics,NaN,NaN,"[{'name': 'povar', 'users': ['U049HDR2Z'], 'co...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67,U34Q3KU8H_1585152153.0
U6QBC2ERG_1585159901.0,639947,message,NaN,1.585160e+09,U6QBC2ERG,"KL в действии, чему тут удивляться. Есть мнени...",ai_ethics,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,511,U6QBC2ERG_1585159901.0
U06J1LG1M_1568636690.0,639948,message,NaN,1.568637e+09,U06J1LG1M,"кароче, эксперты в эйай этике, меня тут позвал...",ai_ethics,NaN,NaN,"[{'name': 'tati', 'users': ['U09JEC7V0', 'UFM7...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,473,U06J1LG1M_1568636690.0


In [37]:
text_dict = {x:y for x,y in zip(data.index, data.text)}

In [38]:
def get_pos_score(reactions):
    positive_reactions = {
        'heavy_plus_sign',
        '+1',
        'tnx',
        'omgtnx',
        'muscle',
        'fireball',
        'shaka',
        'fire',
        'heart'
    }
    total_pos_score = 0
    for reaction in reactions:
        if reaction['name'] in positive_reactions:
            total_pos_score += reaction['count']
    return total_pos_score


pos_score_dict = {}
for row_ind, row in tqdm(data['reactions'].dropna().iteritems()):
    pos_score = get_pos_score(eval(row))
    pos_score_dict[row_ind] = pos_score

In [115]:
def append_to_dict(d:dict, 
                   new_ind, 
                   text,
                   answer_count,
                   answer_text, 
                   text_score, 
                   channel):
    
    d['new_ind'].append(new_ind)
    d['text'].append(text)
    d['answer_count'].append(answer_count)
    d['answer_text'].append(answer_text)
    d['text_score'].append(text_score)
    d['channel'].append(channel)

In [116]:
replies_base = []
fail_count = 0
success_count = 0
for row_id, row in tqdm(data[~data['replies'].isnull()].iterrows()):
    try:
        replies = eval(row['replies'])
        success_count += 1
    except:
        fail_count += 1
    
    for reply in replies:
        new_ind = f'{reply["user"]}_{round(float(reply["ts"]), 0)}'
        replies_base.append(new_ind)

In [117]:
len(replies_base)

625528

In [130]:
from collections import defaultdict
df_dict = {
    'new_ind':[],
    'text': [],
    'answer_count': [],
    'answer_text': [],
    'text_score': [],
    'channel': []
              }

fail_count = 0
success_count = 0
all_index = set(data.index)
replies_dict = defaultdict(list)
for row_id, row in tqdm(data[~data.index.isin(replies_base)].iterrows()):
    text_score = pos_score_dict.get(row_id,0)
    try:
        replies = eval(row['replies'])
        ans_count = len(replies)
    except:
        append_to_dict(df_dict, row_id, row['text'], 0, None, text_score, row['channel'])
        continue
    
    for reply in replies:
        new_ind = f'{reply["user"]}_{round(float(reply["ts"]), 0)}'
        
        if new_ind in all_index:
            replies_dict[row_id].append(new_ind)
            success_count += 1
        else:
            fail_count += 1
        
    if replies_dict[row_id]:
        row_replies = [(x, pos_score_dict.get(x,0)) for x in replies_dict[row_id]]
        best_k_replies = sorted(row_replies, key=lambda x: x[1], reverse=True)[:10]
        
        replies_id_list = [k for k, v in best_k_replies]
        replies_pos_score_list = [v for k, v in best_k_replies]
        answer = ' '.join(text_dict[k] for k in replies_id_list)
        
        append_to_dict(df_dict, row_id, row['text'], ans_count, answer, text_score, row['channel'])

df = pd.DataFrame.from_dict(df_dict)
df.head()

,new_ind,text,answer_count,answer_text,text_score,channel
0,U0106VCFW81_1585131557.0,"Всем привет! Стоит такая задача, есть историия...",7,"Регрессию построить можно, а вот верить, что о...",0,analytiсs
1,U17S9F6KV_1596184224.0,Привет! Такой вопрос. Мы хотим настроить алерт...,17,как вариант из посгреса передаете информацию в...,0,analytiсs
2,UFF1SJ30F_1595870327.0,Всем привет! А кто-нибудь автоматизировал созд...,8,<@U1G303UTW> обламывает копи-пастой заниматься...,0,analytiсs
3,U8DRB30SX_1597401814.0,"Привет! Есть какие-нибудь материалы, статьи ил...",2,а что там делать. выбрал метрику и посчитал.\n...,1,analytiсs
4,UR9M1T5KP_1589979868.0,"Добрый день ,кто может подсказать пжл статьи,н...",10,"много нулей - ну бывает, скошенное распределен...",0,analytiсs


In [131]:
print(success_count, fail_count)

557795 67733


In [132]:
df.shape

(284315, 6)

## Filter short messages

In [133]:
df['text_len'] = df.text.apply(lambda x: len(x.split()))

In [134]:
df = df[df.text_len>5]

In [135]:
df.shape

(236895, 7)

In [136]:
df.head()

,new_ind,text,answer_count,answer_text,text_score,channel,text_len
0,U0106VCFW81_1585131557.0,"Всем привет! Стоит такая задача, есть историия...",7,"Регрессию построить можно, а вот верить, что о...",0,analytiсs,38
1,U17S9F6KV_1596184224.0,Привет! Такой вопрос. Мы хотим настроить алерт...,17,как вариант из посгреса передаете информацию в...,0,analytiсs,27
2,UFF1SJ30F_1595870327.0,Всем привет! А кто-нибудь автоматизировал созд...,8,<@U1G303UTW> обламывает копи-пастой заниматься...,0,analytiсs,43
3,U8DRB30SX_1597401814.0,"Привет! Есть какие-нибудь материалы, статьи ил...",2,а что там делать. выбрал метрику и посчитал.\n...,1,analytiсs,14
4,UR9M1T5KP_1589979868.0,"Добрый день ,кто может подсказать пжл статьи,н...",10,"много нулей - ну бывает, скошенное распределен...",0,analytiсs,33


## No comments, no pluses

In [137]:
df=df[(df['answer_count']>0) | (df['text_score']>0)]

In [138]:
df.head()

,new_ind,text,answer_count,answer_text,text_score,channel,text_len
0,U0106VCFW81_1585131557.0,"Всем привет! Стоит такая задача, есть историия...",7,"Регрессию построить можно, а вот верить, что о...",0,analytiсs,38
1,U17S9F6KV_1596184224.0,Привет! Такой вопрос. Мы хотим настроить алерт...,17,как вариант из посгреса передаете информацию в...,0,analytiсs,27
2,UFF1SJ30F_1595870327.0,Всем привет! А кто-нибудь автоматизировал созд...,8,<@U1G303UTW> обламывает копи-пастой заниматься...,0,analytiсs,43
3,U8DRB30SX_1597401814.0,"Привет! Есть какие-нибудь материалы, статьи ил...",2,а что там делать. выбрал метрику и посчитал.\n...,1,analytiсs,14
4,UR9M1T5KP_1589979868.0,"Добрый день ,кто может подсказать пжл статьи,н...",10,"много нулей - ну бывает, скошенное распределен...",0,analytiсs,33


In [147]:
df.shape

(77155, 7)

In [143]:
df[df['answer_count']==0].shape

(14944, 7)

In [145]:
df.reset_index(drop=True, inplace=True)

In [178]:
df['answer_text'] = df['answer_text'].fillna(" ")

In [179]:
df.to_csv('./data/ods_new_base.csv', index = False)